# Binder discovery from publicly available data
The aim of this project is to develop a semi-automated search for existing binders for particular targets. This approach will combine a sequence-based search to identify scaffolds of interest with a text based search of annotations for the target. 

## Design
### Functionality
Major features to be implimented in this project:

* database selection (pdb, patents, nr– dropdown)
* scaffold selection (possibly dropdown or hmm or pfam id, eg V<sub>H</sub>H/PF07686)
* annotation search (keyword input)

Other features:

* if target is a also a protein, check to see if co-crystal structure exists in pdb
* differentiate V<sub>H</sub>H from S<sub>C</sub>F<sub>V</sub>, FAB, and full-length IgG antibodies (length, presence/absence of constant domains)
* expand text search to include multiple sources (patent contents/associated papers etc.)

### Database selection
This should be the easiest to implement. Download data and set up searchable database. Select by identifier.

### Scaffold selection
This can be done in a couple of ways. If only pre-defined options exist then searches can be pre-computed and a reduced dataset provided for each scaffold selection. If the user provides an hmm or pfam id, this lookup will have to be performed on the fly using a call to hmmsearch.

Selecting for antibody subtypes could be difficult. The v-set pfam ([PF07686](pfam.xfam.org/family/PF07686)) will match any IgG-like variable region. Selecting by domain architectures might be difficult as distinguishing between truncated sequences and true antibody fragments might be impossible. The data annotations may have to be relied upon for this or the burden passed to the user. 

This assumption should be tested by building a curated model of each scaffold of interest and searching against the database, scoring the fidelity with which that specific scaffold is returned. 

### Annotation search
This will be the trickiest. Ideally an advanced search will be used which takes synonyms into account as well as misspellings. Also, what can be searched will depend on which database is being searched. There are vastly different data associated with each of the PDB and patent databases. The best solution might be to use an existing API to leverage the advanced search functionality. Examples might be the [Google search API](https://developers.google.com/custom-search/) together with the patent database flag, "&tbm=pts"; the EMBL-EBI [ENA API](https://www.ebi.ac.uk/about/news/service-news/new-ena-discovery-api); or [pyPDB](https://github.com/williamgilpin/pypdb) a python API for the RCSB PDB.

### Combining filters
#### Patent database
Patents have a unique identifier. This identifier is returned in a list by most text searches. By performing the text search via an API, the list of returned patent applications can be compared against the list of patents containing sequences of interest.  
There are multiple patent resources. EMBL-EBI provoides a database of patent sequences. Google scholar and The Lens provide searches of the patent literature. The Lens also provides some ability to access sequences associated with those patents.

#### PDB database
As above, except the unique identifier is the pdb id. 

#### IMGT antibody database
This is already a highly curated database so all that needs to be done is to perform a text search. 



## Todo
* download the nucleotide patent dataset and compare it to the protein one. Does it contain antibody sequences not contained in the protein set? Is it mainly genomic loci, primers, etc? 
* Download IMGT antibody database
* See if The Lens can be scraped to extract all sequences from a query. 

```bash
# remove spaces in fasta headers so the patent number is carried across to the hmmsearch results 
cat ~/data/databases/nrp_patent.fa | sed -e 's/ /\|/g' > ~/data/databases/nrp_patent_nospace.fa

# get a list of ids for seqs in the patent database containing the vset pfam domain
hmmsearch --domtblout patent_vsets.dtbl -o patent-vset.out ~/working_dir/binder_filter/scfv/V-set.hmm ~/data/databases/nrp_patent_nospace.fa

# build ssi index of seqs
esl-sfetch --index ~/data/databases/nrp_patent_nospace.fa

# extract those sequences from the database (uniq removes duplicate ids when the domain matches multiple locations in a single sequence)
grep -v "^#" patent_vsets.dtbl | awk '{print $1}' | uniq | esl-sfetch -f ~/data/databases/nrp_patent_nospace.fa - > patent-vsets.fa
```

#### Building HMMs from curated sets of antibody scaffolds
In order to test whether antibody structures can be differentiated from sequence alone HMMs were built using curated sets of each type. 

A curated set was produced by accessing the IMGT database and restricting the search terms to the scaffold of interest, for example by selecting "scFv" under "other keywords". This produced 447 sequences, all of which were exported and then aligned using muscle:

```bash
muscle -in imgt-scfvs.fasta -out imgt-scfvs-aligned.fasta
```

The aligned sequences were then visualised in geneious and especially long or truncated sequences were removed. This resulted in 226 full-length ScFv sequences. These were then translated using BioPython and filtered for sequences with internal stop-codons:

```python
from Bio import SeqIO, Seq
h = "/Users/j.parker/working_dir/binder_filter/scfv/imgt-aligned-scfvs_extracts.fasta"

# filter for sequences whose length is not a multiple of three.
with open(h) as f:
    seqs  = [seq for seq in SeqIO.parse(f, "fasta") \
             if (len(seq) % 3 == 0)]

print ("there are {} sequences to be translated".format(len(seqs)))

out = "./scfvs-translated.fa"
translated = []
for seq in seqs:
    t = seq.seq.translate()
    # remove trailing stop-codons for downstream analysis
    t = t.strip("*") 
    # filter for internal stop codons
    if "*" not in t:
        seq.seq = t
        translated.append(seq)
    else:
        pass

print ("there are {} sequences to be written".format(len(translated)))

SeqIO.write(translated, out, "fasta")
```

This resulted in a final set of 205 ScFv sequences. These sequences were realigned using muscle and then used to build an HMM and search for matching sequences in the patent dataset:

```bash
muscle -in scfvs-translated.fa -out scfvs-translated-aligned.fa;
hmmbuild scfv.hmm scfvs-translated-aligned.fa

hmmsearch --domtblout patent_scfvs.dtbl -o ~/working_dir/binder_filter/scfv/patent-scfvs.out ~/working_dir/binder_filter/scfv/scfv.hmm ~/data/databases/nrp_patent_nospace.fa
```

This HMM was then used to search the PDB and the results visually checked to make sure the significant results were predominantly ScFv sequences and not V<sub>H</sub>H or full-length monoclonal antibodies. Approximately 80% of the sequences were ScFvs but there were a number of diabodies as well. This is probably not a problem or surprising as diaboides are similar to ScFvs with different connectivity, and is a burden that can be shifted to the user.

The above process of extracting sequences from the patent dataset was used for both the ScFv and V<sub>H</sub>H HMMs.

For the V<sub>H</sub>Hs, a length filter of 250aa was also applied to exclude full-length and longer fragment antibodies which reduced the dataset from 140061 to 118448 sequences.

```python
from Bio import SeqIO, Seq
h = "/Users/j.parker/working_dir/binder_filter/vhh/patent-vhhs.fa"
out = "./patent-vhhs-filtered.fa"

# filter for sequences whose length is not a multiple of three.
with open(h) as f:
    seqs  = [seq for seq in SeqIO.parse(f, "fasta") \
             if (len(seq) <= 250)]
             
SeqIO.write(seqs, out, "fasta")

```

The patent numbers were then extracted from the fasta headers and saved as a separate file:

```bash
grep ">" ~/working_dir/binder_filter/vhh/patent-vhhs-filtered.fa | awk -F "|" '{print $2}' | awk -F ":" '{print $2}' > ~/working_dir/binder_filter/vhh/patent-vhhs-filtered.ids
```

## Test-case: V-set and the Patent database

In [117]:
import pandas as pd
import re

test_json = "/Users/j.parker/working_dir/binder_filter/text_searches/testosterone-lens-export.json"
test_csv = "/Users/j.parker/working_dir/binder_filter/text_searches/lens-export.csv"

In [168]:
df = pd.read_csv(test_csv, index_col="#")
df["Publication Number"] = ["".join(re.split(" |/", pn)[:-1]) for pn in df["Publication Number"]]

df.head()

,Jurisdiction,Kind,Publication Number,Lens ID,Publication Date,Publication Year,Application Number,Application Date,Priority Numbers,Title,...,Cited Count,Simple Family Size,Extended Family Size,Sequence Count,CPC Classifications,IPCR Classifications,US Classifications,PubMed Id(s),DOI(s),Non-Patent Citations
#,,,,,,,,,,,,,,,,,,,,,
1,WO,A2,WO2008009960,093-144-008-355-452,2008-01-24,2008,GB 2007002764 W,2007-07-20,GB 0614568 A 20060721,Anti-testosterone Antibodies,...,5,4,4,20,C07K16/26;;C07K2317/21;;C07K2317/76,C07K16/00,NaN,9661815,10.1016/S1380-2933(98)00002-5,NaN
2,US,A1,US20060258632,066-185-105-955-080,2006-11-16,2006,US 41840206 A,2006-05-04,EP 02090315 A 20020904,Methods And Agents For Treating Cardiovascular...,...,1,10,10,103,A61K31/56,A61K31/57;;A61K31/56;;A61P9/00,514/177.000,NaN,NaN,NaN
3,US,A1,US20040110733,120-693-751-095-318,2004-06-10,2004,US 30956202 A,2002-12-04,EP 02090315 A 20020904,Methods And Agents For Treating Cardiovascular...,...,5,10,10,2,A61K31/56,A61K31/56;;A61P9/00,514/171,NaN,NaN,NaN
4,US,B1,US7595056,099-114-088-850-867,2009-09-29,2009,US 44963403 A,2003-05-30,US 38424902 P 20020530,Methods Of Rejuvenating Leydig Cells And Enhan...,...,1,1,1,6,A61K38/24;;A61K2300/00,A61K39/00;;A01N37/18,424/198.1;;514/2,10673146;;10993816;;11113606;;11133664;;112592...,10.1095/BIOLREPROD64.4.1273;;10.1095/BIOLREPRO...,Elmlinger et al. 2003 Clin Chem Lab Med. 41:93...
5,US,B2,US7138246,059-214-848-675-051,2006-11-21,2006,US 6860605 A,2005-02-28,US 6860605 A 20050228;;US 54885104 P 20040301;...,Methods For Identifying Or Screening For Agent...,...,1,4,19,3,G01N33/743,G01N33/53;;G01N33/567;;G01N33/74,435/7.1;;435/7.2;;435/7.21;;436/63;;436/86;;43...,10926083;;11806715;;2149506;;2821659;;6574004;...,10.1016/0960-0760(90)90441-M;;10.1038/NG0594-3...,Fiet et al. (Clinical Chemistry 40(12) 2296-23...


In [169]:
vset_ids = pd.read_csv('/Users/j.parker/working_dir/binder_filter/vset/patent-vsets.ids',
                     header=None, names=["Publication Number"], index_col="Publication Number")

In [178]:
vset_patents = df[df["Publication Number"].isin(vset_ids.index)]
print ("Number of patents containing vset domain in sequence: {}".format(len(vset_patents)))
vset_patents.head()

Number of patents containing vset domain in sequence: 4


,Jurisdiction,Kind,Publication Number,Lens ID,Publication Date,Publication Year,Application Number,Application Date,Priority Numbers,Title,...,Cited Count,Simple Family Size,Extended Family Size,Sequence Count,CPC Classifications,IPCR Classifications,US Classifications,PubMed Id(s),DOI(s),Non-Patent Citations
#,,,,,,,,,,,,,,,,,,,,,
1,WO,A2,WO2008009960,093-144-008-355-452,2008-01-24,2008,GB 2007002764 W,2007-07-20,GB 0614568 A 20060721,Anti-testosterone Antibodies,...,5,4,4,20,C07K16/26;;C07K2317/21;;C07K2317/76,C07K16/00,NaN,9661815,10.1016/S1380-2933(98)00002-5,NaN
12,WO,A3,WO2008009960,052-777-162-555-746,2008-07-10,2008,GB 2007002764 W,2007-07-20,GB 0614568 A 20060721,Anti-testosterone Antibodies,...,0,4,4,20,C07K16/26;;C07K2317/21;;C07K2317/76,C07K16/26;;A61K39/395;;C12N15/13,NaN,NaN,NaN,"HEMMINKI A ET AL: ""Fine tuning of an anti-test..."
32,WO,A9,WO2008009960,128-391-507-152-525,2008-05-29,2008,GB 2007002764 W,2007-07-20,GB 0614568 A 20060721,Anti-testosterone Antibodies,...,0,4,4,20,C07K16/26;;C07K2317/21;;C07K2317/76,C07K16/26;;A61K39/395;;C12N15/13,NaN,NaN,NaN,NaN
68,US,A,US5977319,134-667-134-506-117,1999-11-02,1999,US 95820197 A,1997-10-21,US 95820197 A 19971021;;US 2889796 P 19961021,Specific Binding Members For Estradiol; Materi...,...,12,1,1,23,C07K16/26;;C07K2317/34;;C07K2317/622,C07K16/26,530/388.24;;424/145.1,1367224;;1404388;;1565653;;1748994;;1834458;;1...,10.1016/0022-2836(92)90223-7;;10.1016/S1380-29...,"Aravelo, J.H. et al., Three dimensional Struct..."


In [179]:
vhh_ids = pd.read_csv('/Users/j.parker/working_dir/binder_filter/vhh/patent-vhhs-filtered.ids',
                     header=None, names=["Publication Number"])
vhh_patents = df[df["Publication Number"].isin(vhh_ids["Publication Number"])]
print ("Number of patents containing VHH domain in sequence: {}".format(len(vhh_patents)))
vhh_patents.head()

Number of patents containing VHH domain in sequence: 4


,Jurisdiction,Kind,Publication Number,Lens ID,Publication Date,Publication Year,Application Number,Application Date,Priority Numbers,Title,...,Cited Count,Simple Family Size,Extended Family Size,Sequence Count,CPC Classifications,IPCR Classifications,US Classifications,PubMed Id(s),DOI(s),Non-Patent Citations
#,,,,,,,,,,,,,,,,,,,,,
1,WO,A2,WO2008009960,093-144-008-355-452,2008-01-24,2008,GB 2007002764 W,2007-07-20,GB 0614568 A 20060721,Anti-testosterone Antibodies,...,5,4,4,20,C07K16/26;;C07K2317/21;;C07K2317/76,C07K16/00,NaN,9661815,10.1016/S1380-2933(98)00002-5,NaN
12,WO,A3,WO2008009960,052-777-162-555-746,2008-07-10,2008,GB 2007002764 W,2007-07-20,GB 0614568 A 20060721,Anti-testosterone Antibodies,...,0,4,4,20,C07K16/26;;C07K2317/21;;C07K2317/76,C07K16/26;;A61K39/395;;C12N15/13,NaN,NaN,NaN,"HEMMINKI A ET AL: ""Fine tuning of an anti-test..."
32,WO,A9,WO2008009960,128-391-507-152-525,2008-05-29,2008,GB 2007002764 W,2007-07-20,GB 0614568 A 20060721,Anti-testosterone Antibodies,...,0,4,4,20,C07K16/26;;C07K2317/21;;C07K2317/76,C07K16/26;;A61K39/395;;C12N15/13,NaN,NaN,NaN,NaN
68,US,A,US5977319,134-667-134-506-117,1999-11-02,1999,US 95820197 A,1997-10-21,US 95820197 A 19971021;;US 2889796 P 19961021,Specific Binding Members For Estradiol; Materi...,...,12,1,1,23,C07K16/26;;C07K2317/34;;C07K2317/622,C07K16/26,530/388.24;;424/145.1,1367224;;1404388;;1565653;;1748994;;1834458;;1...,10.1016/0022-2836(92)90223-7;;10.1016/S1380-29...,"Aravelo, J.H. et al., Three dimensional Struct..."


In [180]:
scfv_ids = pd.read_csv('/Users/j.parker/working_dir/binder_filter/scfv/patent-scfvs-filtered.ids',
                      header=None, names=["Publication Number"])
scfv_patents = df[df["Publication Number"].isin(vhh_ids["Publication Number"])]
print ("Number of patents containing ScFv domain in sequence: {}".format(len(scfv_patents)))
scfv_patents.head()

Number of patents containing ScFv domain in sequence: 4


,Jurisdiction,Kind,Publication Number,Lens ID,Publication Date,Publication Year,Application Number,Application Date,Priority Numbers,Title,...,Cited Count,Simple Family Size,Extended Family Size,Sequence Count,CPC Classifications,IPCR Classifications,US Classifications,PubMed Id(s),DOI(s),Non-Patent Citations
#,,,,,,,,,,,,,,,,,,,,,
1,WO,A2,WO2008009960,093-144-008-355-452,2008-01-24,2008,GB 2007002764 W,2007-07-20,GB 0614568 A 20060721,Anti-testosterone Antibodies,...,5,4,4,20,C07K16/26;;C07K2317/21;;C07K2317/76,C07K16/00,NaN,9661815,10.1016/S1380-2933(98)00002-5,NaN
12,WO,A3,WO2008009960,052-777-162-555-746,2008-07-10,2008,GB 2007002764 W,2007-07-20,GB 0614568 A 20060721,Anti-testosterone Antibodies,...,0,4,4,20,C07K16/26;;C07K2317/21;;C07K2317/76,C07K16/26;;A61K39/395;;C12N15/13,NaN,NaN,NaN,"HEMMINKI A ET AL: ""Fine tuning of an anti-test..."
32,WO,A9,WO2008009960,128-391-507-152-525,2008-05-29,2008,GB 2007002764 W,2007-07-20,GB 0614568 A 20060721,Anti-testosterone Antibodies,...,0,4,4,20,C07K16/26;;C07K2317/21;;C07K2317/76,C07K16/26;;A61K39/395;;C12N15/13,NaN,NaN,NaN,NaN
68,US,A,US5977319,134-667-134-506-117,1999-11-02,1999,US 95820197 A,1997-10-21,US 95820197 A 19971021;;US 2889796 P 19961021,Specific Binding Members For Estradiol; Materi...,...,12,1,1,23,C07K16/26;;C07K2317/34;;C07K2317/622,C07K16/26,530/388.24;;424/145.1,1367224;;1404388;;1565653;;1748994;;1834458;;1...,10.1016/0022-2836(92)90223-7;;10.1016/S1380-29...,"Aravelo, J.H. et al., Three dimensional Struct..."


### Results

All three of these searches returned the same patents. It is possible that the hmmsearch parameteres were too flexible, leading to the inclusion of essentially the same subset of sequences. One possible solution would be to require a certain level of coverage between the HMM profile and the database.  

In [181]:
pwd

'/Users/j.parker/working_dir/binder_filter/scfv'